In [ ]:
from google.genai import types
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.runners import InMemoryRunner
from google.adk.models.google_llm import Gemini
from google.adk.tools.base_tool import BaseTool
from google.adk.agents.base_agent import BaseAgent
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse
from google.adk.plugins.base_plugin import BasePlugin
from google.adk.tools.tool_context import ToolContext
from google.adk.agents.callback_context import CallbackContext

In [2]:
retry_config=types.HttpRetryOptions(attempts=3, exp_base=5, initial_delay=1, http_status_codes=[429, 500, 503, 504])
model = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config)

In [3]:
history_teacher_agent = Agent(
    name="HistoryTeacherAgent",
    model=model,
    instruction="""
    You are a history teacher.
    Answer all my questions about history.
    If you are not sure, use google_search tool.
    """,
    tools=[google_search],
)

In [15]:
class CustomPlugin(BasePlugin):
    async def before_agent_callback(self, *, agent: BaseAgent, callback_context: CallbackContext) -> None:
        print(f"[CustomPlugin] Agent {agent.name} will run")

    async def after_agent_callback(self, *, agent: BaseAgent, callback_context: CallbackContext) -> None:
        print(f"[CustomPlugin] Agent {agent.name} did run")

    async def before_model_callback(self, *, callback_context: CallbackContext, llm_request: LlmRequest) -> None:
        print(f"[CustomPlugin] Model will be called")
    
    async def after_model_callback(self, *, callback_context: CallbackContext, llm_response: LlmResponse) -> None:
        print(f"[CustomPlugin] Model was called")
    
    async def before_tool_callback(self, *, tool: BaseTool, tool_args: dict, tool_context: ToolContext) -> None:
        print(f"[CustomPlugin] Tool {tool.name} will be called with args: {tool_args}")
    
    async def after_tool_callback(self, *, tool: BaseTool, tool_args: dict, tool_context: ToolContext, result: dict) -> None:
        print(f"[CustomPlugin] Tool {tool.name} was called with args: {tool_args} and result: {result}")

In [16]:
runner = InMemoryRunner(agent=history_teacher_agent, plugins=[CustomPlugin(name="CustomPlugin")])
_ = await runner.run_debug("How many times was Donald Trump elected president?")


 ### Created new session: debug_session_id

User > How many times was Donald Trump elected president?
[CustomPlugin] Agent HistoryTeacherAgent will run
[CustomPlugin] Model will be called
[CustomPlugin] Model was called
HistoryTeacherAgent > Donald Trump has been elected president of the United States twice. He served as the 45th president from 2017 to 2021 and began his second, non-consecutive term as the 47th president on January 20, 2025.
He was first elected in 2016, defeating Hillary Clinton. He then unsuccessfully sought reelection in 2020, losing to Joe Biden. In 2024, he was elected to a second term, defeating Kamala Harris.
[CustomPlugin] Agent HistoryTeacherAgent did run
